In [ ]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from read_data import read_world, read_sensor_data
from importlib import reload
import particle_filter 

# Add random seed for generating comparable pseudo random numbers
np.random.seed(123)

print("Reading landmark positions")
landmarks = read_world("./data/world.dat")

print("Reading sensor data")
sensor_readings = read_sensor_data("./data/sensor_data.dat")

In [ ]:
# Allows us to modify the particle filter routines and run 
# without having to reload the Python kernel
reload(particle_filter)

# Initialize the particles
map_limits = [-1, 12, 0, 10]
particles = particle_filter.initialize_particles(1000, map_limits)

# Plot preferences, interactive plotting mode
plt.axis(map_limits)
plt.ion()
# plt.show()

# Run particle filter
assert(len(sensor_readings) % 2 == 0)
for timestep in range(len(sensor_readings) >> 1):
    # Plot the current state
    if timestep % 5 == 0:
        particle_filter.plot_state(particles, landmarks, map_limits)

    # Predict particles by sampling from motion model with odometry info
    new_particles = particle_filter.sample_motion_model(
        sensor_readings[timestep, 'odometry'], particles)

    # Calculate importance weights according to sensor model
    weights = particle_filter.eval_sensor_model(
        sensor_readings[timestep, 'sensor'], new_particles, landmarks)

    # Resample new particle set according to their importance weights
    particles = particle_filter.resample_particles(new_particles, weights)

plt.show('hold')